##  Decorators

In [2]:
def decorate1(func):
    def inner():
        print("apply decorate1")
        func()
    return inner

@decorate1
def hello():
    print("hello, world!")
        
hello()

apply decorate1
hello, world!


In [3]:
%reset -f 
def decorate1(func):
    def inner():
        print("apply decorate1")
        func()
    return inner

def hello():
    print("hello, world!")

hello = decorate1(hello)
hello()

apply decorate1
hello, world!


In [4]:
def decorate2(func):
    def inner():
        print("apply decorate2")
        func()
    return inner

@decorate1
@decorate2
def hello12():
    print("hello, world!")
hello12() # same as hello12 = decorate1(decorate2(hello12))

apply decorate1
apply decorate2
hello, world!


In [5]:
@decorate2
@decorate1
def hello21():
    print("hello, world!") 
hello21() # same as hello21 = decorate2(decorate1(hello21))

apply decorate2
apply decorate1
hello, world!


###  How to pass arguments to the inner function

In [1]:
#adapted from Fluent Python
import functools
def args_to_string(*args,**kw):
    arg_str = []
    if args:
        arg_str.append(','.join(str(arg) for arg in args))
    if kw:
        arg_str.append(', '.join(('{0}={1}'.format(k,v) for k,v in kw.items())))
    return ','.join(a for a in arg_str)

import time
def time_this(func):
    def decorated(*args,**kw):
        t0 = time.perf_counter()
        result = func(*args,**kw)
        t1 = time.perf_counter()
        name = func.__name__
        arg_str = args_to_string(*args,**kw)
        #print('{0}({1}): [{2}]'.format(name, arg_str,t1-t0))
        #print('{}({}): [{}]'.format(name, arg_str,t1-t0))
        print('%s(%s): [%0.8fs]' % (name, arg_str, t1-t0))
        return result
    return decorated

@time_this
def wait(seconds):
    time.sleep(seconds)

@functools.lru_cache() # <-- note () # parametrized decorators
@time_this
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

@time_this
def sum(a,b):
    return a+b

@time_this
def dummy(*args, **kw):
    a = args
    b = kw

wait(0.3)
factorial(10)
sum(4,5)
dummy('pos', 'second', a='a', b='b')

wait(0.3): [0.30087513s]
factorial(1): [0.00000166s]
factorial(2): [0.00018920s]
factorial(3): [0.00027036s]
factorial(4): [0.00033430s]
factorial(5): [0.00039679s]
factorial(6): [0.00045097s]
factorial(7): [0.00052446s]
factorial(8): [0.00058604s]
factorial(9): [0.00063834s]
factorial(10): [0.00071184s]
sum(4,5): [0.00000282s]
dummy(pos,second,a=a, b=b): [0.00000404s]


In [2]:
factorial(10)
factorial(4)
factorial(11)

factorial(11): [0.00000252s]


39916800

In [3]:
import time
def parametrized_time_this(check=True):
    def decorator(func):
        if not check:
            return func
        def decorated(*args,**kw):
            t0 = time.perf_counter()
            result = func(*args,**kw)
            t1 = time.perf_counter()
            name = func.__name__
            arg_str = args_to_string(*args,**kw)
            print('%s(%s): [%0.8fs]' % (name, arg_str, t1-t0))
            return result
        return decorated
    return decorator # <-- returns the actual decorator
    
@parametrized_time_this(True)
def wait(seconds):
    print('going to sleep for', seconds,'seconds')
    time.sleep(seconds)
    print('woke up!')

wait(0.4)

going to sleep for 0.4 seconds
woke up!
wait(0.4): [0.40102871s]


### Decorators as function objects

In [4]:
import time
class TimeThis():
    def __init__(self, func):           # <--
        self._func = func               # <--
        
    def __call__(self, *args, **kw):
        t0 = time.perf_counter()
        result = self._func(*args,**kw) # <--
        t1 = time.perf_counter()
        name = self._func.__name__      # <--
        arg_str = args_to_string(*args,**kw)
        print('%s(%s): [%0.8fs]' % (name, arg_str, t1-t0))
        return result

@TimeThis
def wait(seconds):
    print('going to sleep for', seconds,'seconds')
    time.sleep(seconds)
    print('woke up!')

wait(0.4)

going to sleep for 0.4 seconds
woke up!
wait(0.4): [0.40096530s]


In [5]:
def func(*args,**kw):
    pass

func = TimeThis(func)
func(0.5)

func(0.5): [0.00000083s]


In [6]:
class ParametrizedTimeThis():
    def __init__(self, check=True):
        self.check = check
    def __call__(self,func):
        if self.check:
            #return TimeThis(func)
            @TimeThis
            def wrapper(*args,**kwargs):
                return func(*args,**kwargs)
            return wrapper
        return func
        
@ParametrizedTimeThis(True)
def wait(seconds):
    print('going to sleep for', seconds,'seconds')
    time.sleep(seconds)
    print('woke up!')

wait(0.4)

going to sleep for 0.4 seconds
woke up!
wrapper(0.4): [0.40100239s]


In [7]:
PTT = ParametrizedTimeThis(True)

def dummy(*args,**kw):
    pass

dummy = PTT(dummy)
dummy(0.4)

wrapper(0.4): [0.00000152s]
